# PART3: Multilabel-Classification using scikit-learn Linear models

## Load Data

In [22]:
import numpy as np
import pandas as pd 

#data_root = '/media/sf_Yelp/input/'
data_root = '/home/long/Desktop/Yelp/input/'

train_photos = pd.read_csv(data_root+'train_photo_to_biz_ids.csv')
train_photo_to_biz = pd.read_csv(data_root+'train_photo_to_biz_ids.csv', index_col='photo_id')

train_df = pd.read_csv(data_root+"train_biz_fc7features.csv")
test_df  = pd.read_csv(data_root+"test_biz_fc7features.csv")

y_train = train_df['label'].values
X_train = train_df['feature vector'].values
X_test = test_df['feature vector'].values

def convert_label_to_array(str_label):
    str_label = str_label[1:-1]
    str_label = str_label.split(',')
    return [int(x) for x in str_label if len(x)>0]

def convert_feature_to_vector(str_feature):
    str_feature = str_feature[1:-1]
    str_feature = str_feature.split(',')
    return [float(x) for x in str_feature]

y_train = np.array([convert_label_to_array(y) for y in train_df['label']])
X_train = np.array([convert_feature_to_vector(x) for x in train_df['feature vector']])
X_test = np.array([convert_feature_to_vector(x) for x in test_df['feature vector']])


In [23]:
print "X_train: ", X_train.shape
print "y_train: ", y_train.shape
print "X_test: ", X_test.shape
print "train_df:"
print train_df[0:5]
print y_train[0:3]

X_train:  (1996, 4096)
y_train:  (1996,)
X_test:  (10000, 4096)
train_df:
   business                  label  \
0     1000%  (1, 2, 3, 4, 5, 6, 7)   
1     1001%           (0, 1, 6, 8)   
2      100%     (1, 2, 4, 5, 6, 7)   
3     1006%        (1, 2, 4, 5, 6)   
4     1010%              (0, 6, 8)   

                                      feature vector  
0  [0.20032024, 0.44084537, 0.23249489, 0.3600976...  
1  [0.0013769998, 0.59398097, 0.55060995, 0.18394...  
2  [0.11435749, 0.033177156, 0.12572332, 0.539482...  
3  [0.075851507, 0.052600037, 0.059594199, 0.7067...  
4  [0.39024171, 0.28424361, 0.0, 0.1655795, 0.460...  
[[1, 2, 3, 4, 5, 6, 7] [0, 1, 6, 8] [1, 2, 4, 5, 6, 7]]


## Train a SVM using cross-validation and assess performance(F1-score)

In [21]:
from sklearn import svm, datasets
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import xgboost as xgb
from sklearn import linear_model
from sklearn import neighbors
from sklearn import naive_bayes
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import cross_validation
from sklearn.metrics import f1_score

import time
t=time.time()

mlb = MultiLabelBinarizer()
y_ptrain= mlb.fit_transform(y_train)  #Convert list of labels to binary matrix

# bgc = BaggingClassifier(base_estimator = linear_model.LogisticRegressionCV(),
#                         n_estimators=5, max_samples = 0.2, bootstrap  = 0, 
#                          n_jobs=-1, verbose = 1)

classifier = OneVsRestClassifier(linear_model.LogisticRegressionCV( random_state= 0))
#classifier.fit(X_ptrain, y_ptrain)
#y_ppredict = classifier.predict(X_ptest)
y_ppredict = cross_validation.cross_val_predict(classifier, X_train, y_ptrain, cv=2, verbose =1)

print "Time passed: ", "{0:.1f}".format(time.time()-t), "sec"

Time passed:  265.2 sec


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.4min finished


In [22]:
print "Samples of predicted labels (in binary matrix):\n", y_ppredict[0:3]
print "\nSamples of predicted labels:\n", mlb.inverse_transform(y_ppredict[0:3])

Samples of predicted labels (in binary matrix):
[[0 1 1 1 1 1 1 1 0]
 [1 0 0 0 0 0 0 0 1]
 [0 1 1 0 1 1 1 1 0]]

Samples of predicted labels:
[(1, 2, 3, 4, 5, 6, 7), (0, 8), (1, 2, 4, 5, 6, 7)]


In [23]:
statistics = pd.DataFrame(columns=[ "attribuite "+str(i) for i in range(9)]+['num_biz'], index = ["biz count", "biz ratio"])
statistics.loc["biz count"] = np.append(np.sum(y_ppredict, axis=0), len(y_ppredict))
pd.options.display.float_format = '{:.0f}%'.format
statistics.loc["biz ratio"] = statistics.loc["biz count"]*100/len(y_ppredict) 
statistics

,attribuite 0,attribuite 1,attribuite 2,attribuite 3,attribuite 4,attribuite 5,attribuite 6,attribuite 7,attribuite 8,num_biz
biz count,565,972,999,1035,505,1239,1379,569,1244,1996
biz ratio,28%,49%,50%,52%,25%,62%,69%,29%,62%,100%


In [24]:
from sklearn.metrics import f1_score

print "F1 score: ", f1_score(y_ptrain, y_ppredict, average='micro') 
print "Individual Class F1 score: ", f1_score(y_ptrain, y_ppredict, average=None)

F1 score:  0.824536875218
Individual Class F1 score:  [ 0.66990291  0.8264631   0.87506173  0.64867517  0.78136882  0.88263666
  0.93026652  0.73093777  0.88960516]


## Re-Train a SVM using all training data, and make predictions on test set

In [24]:
## Uncomment if skip previous train
from sklearn import svm
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import time
import xgboost as xgb
from sklearn.svm import LinearSVC
from sklearn import linear_model
#from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

t = time.time()

mlb = MultiLabelBinarizer()
y_train= mlb.fit_transform(y_train)  #Convert list of labels to binary matrix

from sklearn.cross_validation import StratifiedKFold
print train_df['label'].shape
skf = StratifiedKFold(train_df['label'], n_folds=5, random_state = 0)
y_predict = np.zeros((X_test.shape[0],9))
for train_index, test_index in skf:
    X_train_i = X_train[train_index]
    y_train_i = y_train[train_index]
    classifier = OneVsRestClassifier(linear_model.SGDRegressor(alpha=.0001, n_iter=500,penalty="elasticnet"))
    classifier.fit(X_train_i, y_train_i)
    y_predict_i = classifier.predict(X_test)
    y_predict = y_predict + y_predict_i
    print X_train_i.shape
    print y_train_i.shape
    print y_predict_i[1:3]

y_predict = y_predict/5
#y_predict = classifier.predict_proba(X_test)
np.savetxt(data_root+"../submissions/ensemble/y_predict_skf_SGD_submission_fc7_4096.csv", y_predict, delimiter=",")
y_predict = y_predict > 0.5

#print list(mlb.classes_)
y_predict_label = mlb.inverse_transform(y_predict) #Convert binary matrix back to labels

print "Time passed: ", "{0:.1f}".format(time.time()-t), "sec"

(1996,)
(1500, 4096)
(1500, 9)
[[1 0 0 0 0 1 1 0 0]
 [0 0 0 1 0 0 1 1 1]]
(1571, 4096)
(1571, 9)
[[0 1 1 0 0 1 0 1 0]
 [0 1 1 0 0 0 0 0 0]]
(1617, 4096)
(1617, 9)
[[1 1 0 1 1 0 1 0 0]
 [0 0 1 1 1 0 1 0 1]]
(1640, 4096)
(1640, 9)
[[0 0 0 0 0 1 0 0 0]
 [0 0 1 0 1 1 1 0 0]]
(1656, 4096)
(1656, 9)
[[0 1 1 0 0 1 0 0 0]
 [0 0 1 0 0 1 0 0 0]]
Time passed:  1020.9 sec


In [25]:
print y_predict_label[1:3]

[(1, 5), (2, 6)]


In [26]:
test_data_frame  = pd.read_csv(data_root+"test_biz_fc7features.csv")
df = pd.DataFrame(columns=['business_id','labels'])

for i in range(len(test_data_frame)):
    biz = test_data_frame.loc[i]['business']
    label = y_predict_label[i]
    label = str(label)[1:-1].replace(",", " ")
    df.loc[i] = [str(biz), label]
    
    
with open(data_root+"../submissions/skf_SGD_submission_fc7_4096.csv",'w') as f:
    df.to_csv(f, index=False)    
    

In [27]:
statistics = pd.DataFrame(columns=[ "attribuite "+str(i) for i in range(9)]+['num_biz'], index = ["biz count", "biz ratio"])
statistics.loc["biz count"] = np.append(np.sum(y_predict, axis=0), len(y_predict))
pd.options.display.float_format = '{:.0f}%'.format
statistics.loc["biz ratio"] = statistics.loc["biz count"]*100/len(y_predict) 
statistics

,attribuite 0,attribuite 1,attribuite 2,attribuite 3,attribuite 4,attribuite 5,attribuite 6,attribuite 7,attribuite 8,num_biz
biz count,3657,3843,4105,3248,3485,3673,4964,4470,4119,10000
biz ratio,37%,38%,41%,32%,35%,37%,50%,45%,41%,100%


## Ensemble of previous models

In [11]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import MultiLabelBinarizer

#data_root = '/media/sf_Yelp/input/'
data_root = '/home/long/Desktop/Yelp/input/'
mlb = MultiLabelBinarizer()
y_train= mlb.fit_transform(y_train)  #Convert list of labels to binary matrix


y_predict_BR  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_skf_BR_submission_fc7_4096.csv",header=None)
y_predict_LoR  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_0.824536875218_LogisticRegressionCV_submission_fc7_4096.csv",header=None)
y_predict_XGB  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_skf_XGB_submission_fc7_4096.csv",header=None)
y_predict_RF  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_0.807488855869_RF_d4_n1000_submission_fc7_4096.csv",header=None)
y_predict_RCcv  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_skf_RCcv_norm_submission_fc7_4096.csv",header=None)
y_predict_RC  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_0.820068600663_RC_norm_submission_fc7_4096.csv",header=None)
y_predict_PAC  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_skf_PAC_submission_fc7_4096.csv",header=None)
#y_predict_Per  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_skf_Perceptron_submission_fc7_4096.csv",header=None)

#y_predict_LR  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_0.762498569958_LR_submission_fc7_4096.csv",header=None)


ensemble_predict = (y_predict_BR + y_predict_LoR + y_predict_XGB + y_predict_RF+ y_predict_RCcv+ 
                    y_predict_RC+ y_predict_PAC )/7
ensemble_predict = ensemble_predict >0.5
ensemble_predict = pd.DataFrame.as_matrix(ensemble_predict)
print "Samples of ensemble predict labels (in binary matrix):\n", ensemble_predict[0:3]
#print list(mlb.classes_)
y_predict_label = mlb.inverse_transform(ensemble_predict) #Convert binary matrix back to labels
print "Samples of y_predict_label (in binary matrix):\n", y_predict_label[0:3]


test_data_frame  = pd.read_csv(data_root+"test_biz_fc7features.csv")
df = pd.DataFrame(columns=['business_id','labels'])

for i in range(len(test_data_frame)):
    biz = test_data_frame.loc[i]['business']
    label = y_predict_label[i]
    label = str(label)[1:-1].replace(",", " ")
    df.loc[i] = [str(biz), label]

with open(data_root+"../submissions/ensemble8_7models_skf_submission_fc7_4096.csv",'w') as f:
    df.to_csv(f, index=False)  
    
    

Samples of ensemble predict labels (in binary matrix):
[[False  True  True  True False  True  True False False]
 [False  True  True  True False  True  True False  True]
 [False False False  True False  True  True False  True]]
Samples of y_predict_label (in binary matrix):
[(1, 2, 3, 5, 6), (1, 2, 3, 5, 6, 8), (3, 5, 6, 8)]


## Bagging 